In [1]:
import torch
from torch import nn

import torchvision
from torchvision import transforms

In [6]:
from torch_vision.classification import multi
from torch_vision import dataloader, save_model, set_seed, train_time, walk_through_dir, writer

In [7]:
dir = "../data/data"

In [8]:
walk_through_dir.walk_through_dir(dir)

There are 2 directories and 0 images in '../data/data'.
There are 29 directories and 0 images in '../data/data\test'.
There are 0 directories and 90 images in '../data/data\test\A'.
There are 0 directories and 90 images in '../data/data\test\B'.
There are 0 directories and 90 images in '../data/data\test\C'.
There are 0 directories and 90 images in '../data/data\test\D'.
There are 0 directories and 90 images in '../data/data\test\del'.
There are 0 directories and 90 images in '../data/data\test\E'.
There are 0 directories and 90 images in '../data/data\test\F'.
There are 0 directories and 90 images in '../data/data\test\G'.
There are 0 directories and 90 images in '../data/data\test\H'.
There are 0 directories and 90 images in '../data/data\test\I'.
There are 0 directories and 90 images in '../data/data\test\J'.
There are 0 directories and 90 images in '../data/data\test\K'.
There are 0 directories and 90 images in '../data/data\test\L'.
There are 0 directories and 90 images in '../dat

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [10]:
train_dir = "../data/data/train"
test_dir = "../data/data/test"

In [11]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
auto_transfer = weights.transforms()

In [12]:
auto_transfer

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [14]:
train_dataloader, test_dataloader, class_names = dataloader.create_dataloaders(train_dir=train_dir,
                                                                    test_dir=test_dir,
                                                                    train_transform=auto_transfer,
                                                                    test_transform=auto_transfer,
                                                                    batch_size=32)
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x20026f41940>,
 ['A',
  'B',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H',
  'I',
  'J',
  'K',
  'L',
  'M',
  'N',
  'O',
  'P',
  'Q',
  'R',
  'S',
  'T',
  'U',
  'V',
  'W',
  'X',
  'Y',
  'Z',
  'del',
  'nothing',
  'space'])

In [15]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [16]:
from torchinfo import summary

summary(model=model,
        input_size=[32,3,224,224],
        verbose=0,
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

In [17]:
for param in model.features.parameters():
    param.requires_grad = False

In [18]:
summary(model=model,
        input_size=[32,3,224,224],
        verbose=0,
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [19]:
set_seed.set_seeds()

output_shape = len(class_names)

model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280,
                    out_features=output_shape,
                    bias=True)
).to(device)

In [20]:
summary(model=model,
        input_size=[32,3,224,224],
        verbose=0,
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 29]             --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [21]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                            lr=0.01)

In [22]:
set_seed.set_seeds()

epochs = 5

from timeit import default_timer as timer

start_time = timer()

multi.train(model=model,
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            optimizer=optimizer,
            loss_fn=loss_fn,
            epochs=epochs,
            device=device,
            writer=writer.create_writer(experiment_name="ASL_model",
                                        model_name="model",
                                        extra=f"{epochs}_epochs"))

end_time = timer()

train_time.print_train_time(start=start_time, end=end_time, device=device)

[INFO] created SummaryWriter, saving to: runs\2024_06_30_12_27_36\ASL_model\model\5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 3.2521 | train_acc: 12.1824 | test_loss: 3.0989 | test_acc: 27.5745
Epoch: 2 | train_loss: 2.9963 | train_acc: 34.7646 | test_loss: 2.8549 | test_acc: 49.7798
Epoch: 3 | train_loss: 2.7693 | train_acc: 50.5547 | test_loss: 2.6330 | test_acc: 62.1231
Epoch: 4 | train_loss: 2.5544 | train_acc: 61.7336 | test_loss: 2.4324 | test_acc: 67.8904
Epoch: 5 | train_loss: 2.3752 | train_acc: 66.7979 | test_loss: 2.2529 | test_acc: 71.8369

Train time on cuda: 1999.436 seconds


1999.4364662

In [23]:
save_model.save_model(model=model,
                      target_dir=f"models",
                      model_name=f"ASL_transfer_efficient_b0_{epochs}_epochs.pth")

[INFO] Saving model to: models\ASL_transfer_efficient_b0_5_epochs.pth
